In [1]:
from pydrill.client import PyDrill
import os
import json

from pprint import pprint
from dotenv import load_dotenv

In [2]:
def connect(drill_host: str = os.getenv('DRILL_HOST')):
    auth_string = os.getenv('DRILL_USERNAME') + ":" + os.getenv('DRILL_PASSWORD')
    drill = PyDrill(host=drill_host, port=os.getenv('DRILL_PORT'), use_ssl=True, verify_certs=True, auth=auth_string)

    if drill.is_active():
        print(f'connected to: {drill_host} as user {os.getenv("DRILL_USERNAME")}')
        return drill

    print("Did you place the .env file in the current folder? double-check file contents.")
    return None

In [3]:
load_dotenv()
drill = connect(os.getenv('DRILL_HOSTNAME',''))

True

In [5]:
result = drill.query('''SELECT * FROM sys.version''')

In [6]:
pprint(result.__dict__)

{'columns': ['version',
             'commit_id',
             'commit_message',
             'commit_time',
             'build_email',
             'build_time'],
 'data': {'attemptedAutoLimit': 0,
          'columns': ['version',
                      'commit_id',
                      'commit_message',
                      'commit_time',
                      'build_email',
                      'build_time'],
          'metadata': ['VARCHAR',
                       'VARCHAR',
                       'VARCHAR',
                       'VARCHAR',
                       'VARCHAR',
                       'VARCHAR'],
          'queryId': '1d71654a-8d6f-48f4-755f-257736d17108',
          'queryState': 'COMPLETED',
          'rows': [{'build_email': '',
                    'build_time': '22.02.2022 @ 20:20:27 SAST',
                    'commit_id': 'd19878973ef6723250d231258f470340863ddc23',
                    'commit_message': '[maven-release-plugin] prepare release '
                  

In [7]:
#wie viele verschiedene Sensoren (angegeben im Feld serial_number) enthält die Datenmenge?
distinctSensor = drill.query('''SELECT COUNT(DISTINCT COLUMNS[1]) as Sensoren FROM dfs.data.`co2data.tsv` WHERE NOT columns[1] = 'serial_number' ''')

In [8]:
for row in distinctSensor:
    print(row)

{'Sensoren': 22}


In [9]:
#wieviele Datenpunkte je Sensor liegen vor?
datenPunkt = drill.query('''SELECT COLUMNS[1] as serial_number, COUNT(*) as anzahl FROM dfs.data.`co2data.tsv` WHERE NOT columns[1] = 'serial_number' GROUP BY columns[1]''')

In [10]:
for row in datenPunkt:
    print(row)

{'serial_number': 's_8caab57cc961_', 'anzahl': 7}
{'serial_number': 's_3c6105d3abae_', 'anzahl': 2}
{'serial_number': 's_8caab57cc961_284337', 'anzahl': 1131861}
{'serial_number': 's_e8db84c5f33d_', 'anzahl': 13}
{'serial_number': 's_8caab57a6dd9_', 'anzahl': 6}
{'serial_number': 's_d8bfc0147061_283903', 'anzahl': 578456}
{'serial_number': 's_8caab57c3e19_282028', 'anzahl': 1561045}
{'serial_number': 's_e8db84c5f33d_281913', 'anzahl': 2269085}
{'serial_number': 's_e8db84c5f771_300390', 'anzahl': 1665528}
{'serial_number': 's_e8db84c5f33d_0xdeadbeef', 'anzahl': 1515}
{'serial_number': 's_3c6105d3abae_299589', 'anzahl': 1533917}
{'serial_number': 's_8caab57a6dd9_288065', 'anzahl': 2781611}
{'serial_number': 's_8caab57c3e19_', 'anzahl': 1}
{'serial_number': 's_d8bfc014724e_262793', 'anzahl': 2103522}
{'serial_number': 's_8caab57a6dd9', 'anzahl': 11}
{'serial_number': 's_e8db84c5f771_', 'anzahl': 2}
{'serial_number': 's_8caab57a6dd9_0xdeadbeef', 'anzahl': 49}
{'serial_number': 's_10521c01c

In [11]:
#was ist der höchste , und was der niedrigste Temperaturwert?
temperaturWert =  drill.query('''SELECT 
MAX(CAST(CASE isnumeric(COLUMNS[4]) WHEN 0 THEN -999 ELSE CAST(COLUMNS[4] as DOUBLE) END AS DOUBLE)) as max_Temp, 
MIN(CAST(CASE isnumeric(COLUMNS[4]) WHEN 0 THEN -999 ELSE CAST(COLUMNS[4] as DOUBLE) END AS DOUBLE)) as min_Temp 
FROM dfs.data.`co2data.tsv`  WHERE (NOT columns[4] = 'temperature_celsius') 
AND CAST(CASE isnumeric(COLUMNS[4]) WHEN 0 THEN -999 ELSE CAST(COLUMNS[4] as DOUBLE) END AS DOUBLE) <> -999.0 ''',timeout = 30)

In [12]:
for row in temperaturWert:
    print(row)

{'max_Temp': 36.0, 'min_Temp': -1.0}


In [13]:
#Was ist der durchschnittliche Co2-Wert (co2_ppm) je Sensor?
durchschnittlicheCo2-Wert =  drill.query('''SELECT
columns[1] as serial_number,
AVG(CAST(CASE isnumeric(COLUMNS[3]) WHEN 0 THEN -999 ELSE CAST(COLUMNS[3] as INT) END AS INT)) as co2_ppm_avg 
FROM dfs.data.`co2data.tsv`  
WHERE (NOT columns[3] = 'co2_ppm') 
AND CAST(CASE isnumeric(COLUMNS[3]) WHEN 0 THEN -999 ELSE CAST(COLUMNS[3] as INT) END AS INT) <> -999
GROUP BY columns[1]''',timeout = 30)

In [14]:
for row in durchschnittlicheCo2-Wert:
    print(row)

{'serial_number': 's_8caab57cc961_284337', 'co2_ppm_avg': 470.7930161035675}
{'serial_number': 's_e8db84c5f33d_', 'co2_ppm_avg': 488.7692307692308}
{'serial_number': 's_8caab57cc961_', 'co2_ppm_avg': 429.2857142857143}
{'serial_number': 's_3c6105d3abae_', 'co2_ppm_avg': 832.0}
{'serial_number': 's_10521c0202ab_284839', 'co2_ppm_avg': 436.76356589147287}
{'serial_number': 's_e8db84c5f33d', 'co2_ppm_avg': 1403.2289156626507}
{'serial_number': 's_d8bfc0147061_283903', 'co2_ppm_avg': 628.6339168406931}
{'serial_number': 's_8caab57a6dd9_', 'co2_ppm_avg': 983.6666666666666}
{'serial_number': 's_d8bfc014724e_262793', 'co2_ppm_avg': 770.4554948583827}
{'serial_number': 's_8caab57a6dd9_0xdeadbeef', 'co2_ppm_avg': -1.0}
{'serial_number': 's_10521c01cf19_', 'co2_ppm_avg': 467.5}
{'serial_number': 's_e8db84c5f771_300390', 'co2_ppm_avg': 954.0877634759553}
{'serial_number': 's_e8db84c5f33d_0xdeadbeef', 'co2_ppm_avg': -1.0}
{'serial_number': 's_e8db84c5f771_', 'co2_ppm_avg': 863.0}
{'serial_number':

In [15]:
#Wieviele Datenpunkte mit fehlenden Co2-Werte liegen vor?
fehlendenCo2 =  drill.query('''SELECT 
Count(CAST(CASE isnumeric(COLUMNS[3]) WHEN 0 THEN -999 ELSE CAST(COLUMNS[3] as INT) END AS INT)) as co2_ppm_missing 
FROM dfs.data.`co2data.tsv`  
WHERE 
    (NOT columns[3] = 'co2_ppm')
    AND CAST(CASE isnumeric(COLUMNS[3]) WHEN 0 THEN -999 ELSE CAST(COLUMNS[3] as INT) END AS INT) = -999 ''',timeout = 30)

In [16]:
for row in fehlendenCo2:
    print(row)

{'co2_ppm_missing': 19}


In [17]:
#Sowohl relative_humidity_percent, temperature_celcius als auch co2_ppm sollen als auf zwei Nachkommastellen gerundete Fließkommazahl verfügbar sein.
co2_ppm_to_decimal = '''CAST(CASE isnumeric(COLUMNS[3]) WHEN 0 THEN -999.00 ELSE CAST(COLUMNS[3] as DECIMAL(28,2)) END AS  DECIMAL(28,2)) as co2_ppm'''
temperature_celcius_to_decimal = '''CAST(CASE isnumeric(COLUMNS[4]) WHEN 0 THEN -999.00 ELSE CAST(COLUMNS[4] as DECIMAL(28,2)) END AS  DECIMAL(28,2)) as temperature_celcius'''
relative_humidity_percent_to_decimal = '''CAST(CASE isnumeric(COLUMNS[5]) WHEN 0 THEN -999.00 ELSE CAST(COLUMNS[5] as DECIMAL(28,2)) END AS  DECIMAL(28,2)) as relative_humidity_percent'''

query_result = drill.query(f'''SELECT {co2_ppm_to_decimal},{temperature_celcius_to_decimal},{relative_humidity_percent_to_decimal} FROM dfs.data.`co2data.tsv`  WHERE (NOT columns[3] = 'co2_ppm') AND NOT columns[4] = 'temperature_celsius' AND NOT columns[5] = 'relative_humidity_percent' limit 20''',timeout = 1000)

In [18]:
pprint(query_result.__dict__)

{'columns': ['co2_ppm', 'temperature_celcius', 'relative_humidity_percent'],
 'data': {'attemptedAutoLimit': 0,
          'columns': ['co2_ppm',
                      'temperature_celcius',
                      'relative_humidity_percent'],
          'metadata': ['VARDECIMAL(28, 2)',
                       'VARDECIMAL(28, 2)',
                       'VARDECIMAL(28, 2)'],
          'queryId': '1d716509-7ed8-c609-93e5-7d7f5992a084',
          'queryState': 'COMPLETED',
          'rows': [{'co2_ppm': 420.0,
                    'relative_humidity_percent': 36.0,
                    'temperature_celcius': 23.0},
                   {'co2_ppm': 421.0,
                    'relative_humidity_percent': 32.0,
                    'temperature_celcius': 24.0},
                   {'co2_ppm': 420.0,
                    'relative_humidity_percent': 32.0,
                    'temperature_celcius': 24.0},
                   {'co2_ppm': 651.0,
                    'relative_humidity_percent': 44.0,
     